# LLM for unbiased news summarization - testing notebook

### Setup

In [ ]:
!pip install --upgrade langchain-mistralai langchain langchain_core

In [64]:
import os
import getpass
import json
from IPython.display import Markdown

from langchain_mistralai import ChatMistralAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [65]:
if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = 'knCyMDokThthiODC2D1vnnUgEsLYVSt1'

### Model and prompts

In [66]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    max_retries=5,
)

In [67]:
analyst_system_prompt = SystemMessage(
    content="""
Вы являетесь ИИ-экспертом по анализу новостей. \
Ваша работа заключается в анализе новостных источников и представлении информации из них \
в доступной и непредвзятой форме, без каких-либо оценочных суждений, спорных и недоказанных утверждений.\n\
Вам будет представлена подборка новостных сообщений из разных источников по заданной теме. \
Ваша задача будет заключаться в том, чтобы извлечь из них информацию, следуя следующим правилам:\n\
1. Вы предоставляете только подтверждённую информацию из новостей.\n\
2. Если в новостном сообщении присутствуют неподтверждённые заявления, Вы пишете про них в подобной форме: \
"Новостной источник/данный человек заявил, что...".\n\
3. Вы представляете информацию в максимально нейтральной форме.\n\
4. Вы не делаете никаких выводов и оценочных суждений по представленной информации.\n\
5. Вы представляете всю информацию, которую можете получить из предоставленных источников, без цензуры.\n\
6. Источники могут быть на разных языках. Вы представляете всю информацию на русском языке.\n\
7. В ответ на запрос пользователя Вы выводите только информацию из новостей простым текстом \
без форматирования и дополнительных комментариев."""
)

In [68]:
summarizer_system_prompt = SystemMessage(
    content="""Вы являетесь ИИ-агентом, суммаризирующим новостную информацию.\n\
Вам будет представлена представлена информация по определённой теме, собранная из различных \
новостных источников и представленная в сыром виде без оценочных суждений. \
Вашей задачей будет суммаризировать представленную информацию, чтобы представить её в структурированном, \
удобном для чтения пользователем виде. В ответ на запрос пользователя Вы выводите только сводку \
представленной информации без дополнительных комментариев."""
)

In [69]:
def extraction_prompt(user_query, news_posts):
    '''
    Create prompt for inforamtion extraction by analyst agent.
    '''
    prompt = f"Извлеките информацию из следующих новостных сообщений по запросу \"{user_query}\":"
    for i, post in enumerate(news_posts):
        prompt += f"""
Сообщение {i+1}:
- Дата и время: {post['date']}.
- Текст: "{post['text']}".
"""

    return HumanMessage(content=prompt)

In [70]:
def summarization_prompt(user_query, info):
    '''
    Create prompt for summarization agent from given user request and extracted info.
    '''
    return HumanMessage(
        content=f"Составьте сводку следующей новостной информации по запросу \"{user_query}\":\n{info}"
    )

In [84]:
def generate_summary(user_query, news_posts):
    '''
    Generate news summary for given user query with given news posts found by RAG.
    '''
    llm = ChatMistralAI(
        model="mistral-large-latest",
        max_retries=5,
        api_key='knCyMDokThthiODC2D1vnnUgEsLYVSt1'
    )
    
    example_extraction_prompt = info_extraction_prompt(example_request, example_news)
    messages = [
        analyst_system_prompt,
        example_extraction_prompt
    ]
    
    example_info = llm.invoke(messages)

    example_summarization_prompt = summarization_prompt(example_request, example_info)
    messages = [
        summarizer_system_prompt,
        example_summarization_prompt
    ]
    
    example_summary = llm.invoke(messages)
    return Markdown(example_summary.content)

### Example

In [80]:
example_query = "Последние новости из канала РБК"
with open('mychannel_messages.json', 'r') as f:
    example_news = json.load(f)[:10]

In [72]:
example_extraction_prompt = info_extraction_prompt(example_query, example_news)
print(example_extraction_prompt.content)

Извлеките информацию из следующих новостных сообщений по запросу "Последние новости из канала РБК":
Сообщение 1:
- Дата и время: 2025-11-14T12:29:47+00:00.
- Текст: "Генконсульство Израиля в Северной столице по просьбе РБК Петербург прокомментировало гипотезу о еврейском происхождении Чебурашки

Резонансную теорию о том, что персонаж Эдуарда Успенского является евреем, на государственный уровень поднял глава думского комитета по бюджету и налогам Андрей Макаров. В качестве аргумента депутат привел коробку с oranges, в которой Чебурашка, согласно сюжету, прибыл в страну.

🟢Генеральный консул Израиля в Петербурге Ран Гидор напомнил, что данные о родителях Чебурашки и его национальности доподлинно неизвестны, однако культовый статус персонажа в Израиле — неоспорим.

«Герой советского мультика стал культовым персонажем и в Израиле, ведь он, метафорически выражаясь, сделал алию, приехав в нашу страну с волной советской эмиграции, — отметил Ран Гидор. — Выходцы из СССР способствовали не толь

In [73]:
messages = [
    analyst_system_prompt,
    example_extraction_prompt
]

example_info = llm.invoke(messages)
print(example_info.content)

1. Генеральный консул Израиля в Петербурге Ран Гидор прокомментировал гипотезу о еврейском происхождении Чебурашки, выдвинутую главой думского комитета по бюджету и налогам Андреем Макаровым. Гидор отметил, что данные о родителях и национальности персонажа неизвестны, но Чебурашка стал культовым персонажем в Израиле. Он добавил, что герой "сделал алию", приехав в страну с волной советской эмиграции. В генконсульстве заявили, что были бы рады возможности причислить Чебурашку к еврейской нации, но единственный способ это подтвердить — проверить наличие обрезания и убедиться, что он не ест свинину. Ранее аналогичную теорию выдвигала профессор Майя Балакирски-Кац из Института Туро в Нью-Йорке.

2. Европейский Союз на фоне коррупционного скандала с украинской госкомпанией «Энергоатом» хочет получить от Украины гарантии по расходованию будущей финансовой помощи. Мнения европейских партнеров Киева разделились: одни считают разоблачения позитивным знаком, другие требуют конкретных обязательств

In [74]:
example_summarization_prompt = summarization_prompt(example_query, example_info)

In [75]:
messages = [
    summarizer_system_prompt,
    example_summarization_prompt
]

example_summary = llm.invoke(messages)
Markdown(example_summary.content)

### **Сводка последних новостей из канала РБК**

#### **Культура и общество**
1. **Чебурашка и еврейская теория**
   Генеральный консул Израиля в Петербурге Ран Гидор прокомментировал гипотезу депутата Андрея Макарова о еврейском происхождении Чебурашки. Гидор отметил, что данные о национальности персонажа неизвестны, но Чебурашка популярен в Израиле и «сделал алию» с волной советской эмиграции. В генконсульстве пошутили, что единственный способ подтвердить еврейское происхождение — проверить наличие обрезания и отсутствие употребления свинины.

#### **Политика и экономика**
2. **ЕС требует от Украины гарантий против коррупции**
   На фоне скандала с украинской госкомпанией «Энергоатом» Европейский Союз настаивает на гарантиях по расходованию будущей финансовой помощи. Мнения европейских партнёров разделились: одни видят в разоблачениях позитивный знак, другие требуют конкретных обязательств. Один из чиновников назвал выявленную коррупцию «возмутительной».

3. **Египет повышает стоимость виз почти вдвое**
   Парламент Египта одобрил законопроект о повышении стоимости виз с $25 до $45 из-за нового сбора. Федерация туристических палат страны призвала премьера и министра туризма остановить принятие законопроекта.

4. **Атака дронов на Нововоронежскую АЭС**
   Глава «Росатома» Алексей Лихачев сообщил, что несколько энергоблоков Нововоронежской АЭС были отключены от сети в результате атаки беспилотников в ночь с 12 на 13 ноября. Работа АЭС была восстановлена к 11 утра 14 ноября.

5. **Россия опровергла обвинения в вербовке иракцев для войны на Украине**
   Генконсульство России в Эрбиле (Иракский Курдистан) назвало слухами обвинения в вербовке иракцев для участия в боевых действиях на Украине. Дипломаты заявили, что подобные заявления наносят ущерб имиджу России.

6. **Досрочное прекращение полномочий судейских чиновников**
   Совет судей рассмотрел вопрос о досрочном прекращении полномочий бывшего председателя Совета судей Виктора Момотова и ещё 14 членов, а также об освобождении Владислава Иванова от должности гендиректора Судебного департамента при Верховном суде. Ранее Останкинский суд Москвы изъял в доход государства 95 объектов недвижимости Момотова на сумму около 9 млрд рублей.

#### **Экономика и бизнес**
7. **Шринкфляция обошлась россиянам в 1 трлн рублей**
   Союз потребителей оценил потери россиян из-за шринкфляции (уменьшение объёма товара при сохранении цены) и других уловок в 1 трлн рублей. Среди манипуляций также называются указание цены за 100 грамм вместо цены за упаковку и некорректное оформление акционных цен.

8. **Конгресс «РБК – Общественный капитал»**
   26 ноября эксперты обсудят темы социальной ответственности бизнеса, экономики роста, технологий ответственного благополучия, рынков будущего (БРИКС и ШОС), российского стандарта общественного капитала и экологии.

#### **Власть и правосудие**
9. **Совет Безопасности обсудил киберугрозы**
   Владимир Путин провёл оперативное совещание с постоянными членами Совета Безопасности, на котором обсуждалось противодействие правонарушениям с использованием информационно-коммуникационных технологий.

10. **Приговор Леониду Гозману**
   Суд заочно приговорил оппозиционера Леонида Гозмана к десяти годам колонии по делу об оправдании терроризма.

In [85]:
generate_summary(example_query, example_news)

### **Сводка последних новостей из канала РБК**

#### **Культура и общество**
- **Генеральный консул Израиля в Петербурге Ран Гидор** прокомментировал гипотезу депутата Андрея Макарова о еврейском происхождении Чебурашки. Гидор отметил, что национальность персонажа неизвестна, но Чебурашка популярен в Израиле благодаря советской эмиграции. Консульство готово "причислить" его к еврейской нации, если подтвердятся признаки (например, обрезание и отказ от свинины). Ранее подобную теорию выдвигала профессор Майя Балакирски-Кац (Нью-Йорк).

- **Союз потребителей** оценил потери россиян от **шринкфляции** (сокращение объема товара при сохранении цены) и других уловок с ценами в **1 трлн рублей**. Среди манипуляций: указание цены за 100 грамм вместо упаковки, нечеткое оформление акций. Эксперты предложили ввести требование единообразного оформления цен.

#### **Политика и право**
- **Суд в Москве заочно приговорил оппозиционера Леонида Гозмана** (внесен в реестр иноагентов) к **10 годам колонии** по делу об оправдании терроризма.

- **Совет судей рассмотрел досрочное прекращение полномочий** бывшего главы Совета судей Виктора Момотова и еще 14 членов, а также освобождение Владислава Иванова от поста гендиректора Судебного департамента. Результаты не раскрыты. Ранее Останкинский суд Москвы удовлетворил иск Генпрокуратуры об изъятии в доход государства **95 объектов недвижимости** на **9 млрд рублей**.

- **Владимир Путин** провел совещание с членами Совета Безопасности по противодействию правонарушениям с использованием **информационно-коммуникационных технологий**.

#### **Экономика и коррупция**
- **ЕС требует от Украины гарантий** по расходованию будущей финансовой помощи на фоне коррупционного скандала в "Энергоатоме". Часть европейских партнеров считает разоблачения признаком независимости антикоррупционных органов, другие — требуют конкретных обязательств. Один из чиновников назвал выявленную коррупцию "возмутительной".

#### **Международные отношения**
- **Генконсульство России в Эрбиле (Иракский Курдистан) опровергло обвинения** в вербовке иракцев для участия в войне на Украине, назвав их слухами, направленными на ущерб имиджу России. Дипломаты заявили, что у них нет данных о гражданах Ирака, участвующих или погибших в боевых действиях.

- **Парламент Египта одобрил законопроект** о повышении стоимости виз почти вдвое — с **$25 до $45** за счет нового сбора. Федерация туристических палат Египта призвала премьера и министра туризма вмешаться и остановить принятие закона.

#### **Энергетика и безопасность**
- **Глава "Росатома" Алексей Лихачев** сообщил, что несколько энергоблоков **Нововоронежской АЭС** были отключены от сети из-за атаки беспилотников в ночь с 12 на 13 ноября. Станция вернулась на проектную мощность к 11 утра 14 ноября.